In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from src.utils.images import plot_matrix, debug, plot_array_matrix
%autosave 60

Autosaving every 60 seconds


In [4]:
size = 54
count = 10000

In [5]:
from src.supervised_gcal.models import get_lissom
from src.supervised_gcal.optimizers import SequentialOptimizer, CortexHebbian, NeighborsDecay
input_shape = (size, size)
epochs = 1

lgn_size=36
lgn_shape = (lgn_size, lgn_size)
sigma_sorround=0.29540/2 *24
sigma_center=0.07385/2 * 24
lgn_radius = 0.375*size
lgn_factor = 2.33
lgn_params = {'sigma_center':sigma_center, 'sigma_sorround':sigma_sorround, 
                'lgn_factor':lgn_factor, 'radius':lgn_radius}

In [6]:

print(lgn_size)
print(lgn_shape)
print(sigma_sorround)
print(sigma_center)
print(lgn_radius)
print(lgn_factor)
print(lgn_params)

36
(36, 36)
3.5448
0.8862
20.25
2.33
{'sigma_center': 0.8862, 'sigma_sorround': 3.5448, 'lgn_factor': 2.33, 'radius': 20.25}


In [7]:
cortex_size = 48
cortex_shape = (cortex_size, cortex_size)
radius_aff = 0.27083 * 24
radius_excit = 0.10417* cortex_size
radius_inhib = 0.22917*cortex_size
aff_factor = 1.0
excit_factor = 0.9
inhib_factor = 0.9
min_theta = 0.083
max_theta = 0.633
v1_params = {'min_theta':min_theta, 'max_theta':max_theta, 
             'afferent_radius':radius_aff, 'excitatory_radius':radius_excit, 'inhibitory_radius':radius_inhib, 
             'inhib_factor':inhib_factor, 'excit_factor':excit_factor, 'aff_factor':aff_factor, 
             'settling_steps':9}

In [8]:
display(v1_params)

{'aff_factor': 1.0,
 'afferent_radius': 6.49992,
 'excit_factor': 0.9,
 'excitatory_radius': 5.00016,
 'inhib_factor': 0.9,
 'inhibitory_radius': 11.000160000000001,
 'max_theta': 0.633,
 'min_theta': 0.083,
 'settling_steps': 9}

In [9]:
lissom, _, _ = get_lissom(size, lgn_size, cortex_size, lgn_params='lgn_orientations', rlissom_params='rlissom_orientations')

In [11]:
display(lissom)

Lissom (
  (v1): ReducedLissom (
    (inhibitory_module): Cortex (2304 -> 2304, sigma=22.8803328, radius=11.000160000000001)
    (excitatory_module): Cortex (2304 -> 2304, sigma=3.9001248000000004, radius=5.00016)
    (afferent_module): Cortex (1296 -> 2304, sigma=4.999938461538462, radius=6.49992)
    (piecewise_sigmoid): PiecewiseSigmoid (min_theta=0.083, max_theta=0.633)
  , 1296 -> 2304, settling_steps=9, afferent_strength=1, excitatory_strength=0.9, inhibitory_strength=0.9)
  (off): LGN (
    (diff_of_gaussians): DifferenceOfGaussiansLinear (2916 -> 1296, sigma_surround=3.5448, sigma_center=0.8862, radius=13.5, on=False)
    (strength): *2.33
    (piecewise_sigmoid): PiecewiseSigmoid (min_theta=0.0, max_theta=1.0)
  )
  (on): LGN (
    (diff_of_gaussians): DifferenceOfGaussiansLinear (2916 -> 1296, sigma_surround=3.5448, sigma_center=0.8862, radius=13.5, on=True)
    (strength): *2.33
    (piecewise_sigmoid): PiecewiseSigmoid (min_theta=0.0, max_theta=1.0)
  )
, 2916 -> 2304)

In [15]:
aff_lr = 0.9590
exc_lr = 2.55528
inh_lr = 1.80873
optimizer = SequentialOptimizer(
                CortexHebbian(cortex=lissom.v1.afferent_module, learning_rate=aff_lr),
                CortexHebbian(cortex=lissom.v1.excitatory_module, learning_rate=exc_lr),
                CortexHebbian(cortex=lissom.v1.inhibitory_module, learning_rate=inh_lr))

In [16]:
from src.utils.pipeline import Pipeline
from torch.utils.data import Dataset, DataLoader
train_loader = DataLoader(gaussians_inputs, shuffle=True, pin_memory=True)
pipe = Pipeline(lissom, optimizer, cuda=True, log_interval=100)

NameError: name 'gaussians_inputs' is not defined

In [ ]:
from tqdm import tqdm_notebook as tqdm
for epoch in tqdm(range(epochs)):
    pipe.train(train_loader, epoch)

In [ ]:
from src.utils.datasets import CKDataset
size = 96
ck_dataset = CKDataset()
from torch.utils.data import DataLoader
train_loader = DataLoader(ck_dataset, shuffle=True, pin_memory=True)

In [ ]:
from src.utils.pipeline import Pipeline
lissom.cuda()
for inp, _ in train_loader:
    inp = torch.autograd.Variable(inp)
    inp = Pipeline.process_input(inp)
    inp = inp.cuda()
    act = lissom(inp)
    plot_matrix(np.reshape(act.data.cpu().numpy(), cortex_shape))
    break

In [18]:
import src.supervised_gcal.orientation_maps as om

keys_arrays = generate_all_inputs(size, orientations=25)
activations = om.calculate_keys_activations(lissom, keys_arrays)

NameError: name 'generate_all_inputs' is not defined

In [ ]:
from src.utils.images import plot_array_matrix
from src.supervised_gcal.utils.images import images_matrix
plot_array_matrix(map(lambda x:x.view(cortex_size, cortex_size).cpu().numpy(), list(activations.values())))

In [ ]:
from src.supervised_gcal.utils.orientation_maps import get_orientation_map
orientation_map = get_orientation_map(activations, cortex_shape)
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.colorbar()

In [ ]:
from collections import Counter

plt.show()

print("Promedio: {}".format(np.mean(normalized)))
print("Standard deviation: {}".format(np.std(normalized)))